# Na channels: recovery from inactivation

#### **Basic instructions:**
#### 1. **To run code windows/blocks:** 

    - you can either hit the play button to the left of the code window 

    - or you can use they keyboard shortcut: select the block and press 'shift-enter'.

#### 2. **The first time** you run this code notebook, you might get a popup asking to choose which version of Python to use (the python "kernel"). **Just hit enter** to choose the base/default version.

#### 3. Make sure you data (.abf) files are in the "data" folder here on the left. You can just copy/paste the files from where they are saved on your computer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *
update_plot_defaults()

## 1. Choose the data file you want to analyze

#### Put the .abf files with your Na channel recordings in the "data/Na_channels" folder

In [ ]:
data_folder = "data/1-Na_channels"

from glob import glob
data_files = glob(data_folder+"/*.abf")
print(data_folder)
data_files

Choose which file you want to analyze and past the file name here:

In [ ]:
# data_file = 'data/1-Na_channels/nav18-nav_recovery.abf'
data_file = "data/1-Na_channels/inactivation recovery/nav_inactivation_recovery -120mv_3.abf"

Now we can load the file and plot the raw data:

In [ ]:
traces = Trace.from_axon_file(filename=data_file, 
                              load_voltage=True, 
                              load_ttl=True,
                              units=['pA', 'mV', 'V'], 
                              concatenate_sweeps=False)
print(traces)

time_units = 'ms'

# ----------------------------------------------------------------------------------------------------------------
%matplotlib widget
traces.plot(plot_voltage=True, 
            time_units=time_units, 
            sweep='all')  # Options: 'all', or enter a number here if you want to plot individual sweeps
plt.show()

## 2. Signal processing

### 2.1. Optional: apply baseline correction

If your baseline current is not zero, you may need to correct for that to get accurate measurements.

Uncomment this cell block (highlight everything, then cmd+/ on macOS or ctrl+/ on Windows) to run it.

In [ ]:
# Change this to True if you want to subtract the baseline from the sweeps.
subtract_baseline = True
start_baseline = 0
end_baseline = 0.1

In [ ]:
if subtract_baseline:
    traces.subtract_baseline(start_time = start_baseline, 
                             end_time = end_baseline , 
                             time_units = time_units,  # specify seconds (s), or milliseconds (ms)
                             channel = 'current')  # Options: 'current', 'voltage', 'all'
    %matplotlib widget
    ax1, ax2 = traces.plot(plot_voltage=True, plot_ttl=False, time_units=time_units, sweep='all')
    ax1.set_title('After baseline subtraction', y=0.98)
    plt.show()
else:
    print("BASELINE NOT SUBTRACTED")


### 2.2. Optional: apply additional highpass/lowpass filtering

Depending in you recording, you may have 50/60 Hz line noise, high-frequency noise, or drift in your recordings.

The goal here is to only remove the noise with minimal distortion of the data, so be careful not to overdo it

In [ ]:
additional_filtering = False

In [ ]:
if additional_filtering:
    filtered_traces = traces
    # Step 1: Detrend the data to remove linear or constant trends.
    filtered_traces = filtered_traces.detrend(detrend_type='linear', num_segments=1)

    # Step 2: Apply a highpass filter to remove low-frequency noise + lowpass to remove high-frequency noise
    filtered_traces = filtered_traces.filter(
        line_freq=60,    # Frequency (Hz) of electrical noise to remove: 50 Hz (in Europe) or 60 Hz (in the US).
        width=0.5,         # Width (Hz) controls the width of frequency bands around the line frequency the filter cuts out.
        highpass=0.5,   # Removes low-frequency drift. Set a value in Hz (e.g. 1 for 1 Hz).
        lowpass=4900,    # Removes high-frequency noise. Set a value in Hz (e.g. 100 for 100 Hz).
        order=4)         # Controls sharpness of the filter. Higher = sharper cutoff.

    %matplotlib widget
    ax = filtered_traces.plot(plot_voltage=False, plot_ttl=False)
    ax.set_title('After filtering', y=0.98)
    plt.show()

Once you are happy with the filter setting, run the next cell to implement them:

In [ ]:
if additional_filtering:
    traces=filtered_traces

## 3. Measure recovery currents

#### 3.1 Reference current

First let's get the times of all the voltage steps

In [ ]:
step_times = traces.get_step_events(threshold=-50, polarity='positive', sweep='all',
                                   edge='rising', channel='voltage', time_units=time_units)
print(f"Step times:")
step_times

Now we can measure the reference current after the initial step

In [ ]:
window_size = 10 #ms

reference_step_traces = traces.crop(timepoint=step_times[0][0], window=window_size, time_units='ms')
ax = reference_step_traces.plot(plot_voltage=False, plot_ttl=False, time_units=time_units, sweep='all')
ax.set_title('Single stim',y=0.95)
plt.show()

Now we need to define in which time interval we want to measure the current peak

(make sure to pick a start_time that starts after the stim artiface/capacitive transient)

In [ ]:
start_time = 0.2
end_time = 4

reference_currents, _ = traces.get_measurements(start_time=start_time, end_time=end_time, measurement_type='min', time_units=time_units)

#### 3.2. Recovery currents

Now let's repeat this for the other currents

In [ ]:
recovery_currents = []
for i, step_time in enumerate(step_times):
    current,_ = traces.get_measurements(sweep=i, measurement_type='min', time_units=time_units, start_time=step_time[1]+start_time, end_time=step_time[1]+end_time)
    recovery_currents.append(current)

%matplotlib inline
fig,ax = plt.subplots(figsize=(8,3))
x = range(len(recovery_currents))
ax.plot(x, recovery_currents, '-o')
ax.set_ylabel('Recovery current (pA)')
ax.set_xlabel("sweep nr.")
plt.show()

In [ ]:
# Normalize the measured currents against the baseline/reference currents
peak_currents_norm = recovery_currents / reference_currents

### 3.4. Plot the channel recovery curve

Here we will need to manually define the recovery time points based on our protocol.

In [ ]:
# Define the delta t delays for the recovery
interval_step = 5    # milliseconds

end = interval_step*(len(peak_currents_norm)+1)
recovery_time = np.arange(interval_step, end, interval_step)  # in milliseconds

# Plot the normalized current steps against the holding voltages
%matplotlib inline
fig,ax = plt.subplots(figsize=(5, 4))
ax.plot(recovery_time, peak_currents_norm, 'o', color='black', markersize=5)
ax.set_xlabel('Recovery time (ms)')
ax.set_ylabel('Current (norm.)')
ax.set_title('Recovery of NaV currents')

# Fit exponential curve to the data to get the time constant
from scipy.optimize import curve_fit
def exponential_curve(time, V0, tau, V_inf):
    return V0 * np.exp(-time / tau) + V_inf
V0_guess = 0 
Vinf_guess = 1
tau_guess = 20 # ms
p0 = [V0_guess, tau_guess, Vinf_guess]
popt, _ = curve_fit(exponential_curve, recovery_time, peak_currents_norm, p0=p0)
V0_fit, tau_fit, Vinf_fit = popt

print(f"Tau: {tau_fit:.2f} ms")

# Overlay exponential fit
fit_trace = exponential_curve(recovery_time, *popt)
ax.plot(recovery_time, fit_trace, color='r', alpha=1, linewidth=2)
plt.show()